In [20]:
from skimage import feature
from matplotlib import pyplot as plt

import numpy as np
from sklearn.svm import LinearSVC  
from imutils import paths
import argparse
import cv2
from keras.datasets import cifar10


In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [16]:
numPoints = 24
radius = 8
def describe(image, eps=1e-7):
# compute the Local Binary Pattern representation
# of the image, and then use the LBP representation
# to build the histogram of patterns
    lbp = feature.local_binary_pattern(image,numPoints,radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, numPoints + 3), range=(0, numPoints + 2)) 
# # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    # return the histogram of Local Binary Patterns
    return hist

In [17]:
data = []
labels = []
for i in range(0,x_train.shape[0]):
    img = x_train[i]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist = describe(gray)
    labels.append(y_train[i])
    data.append(hist)
#     print(hist.shape)
#     break
np.save('train_data.npy',data)
np.save('train_label.npy',labels)    

In [21]:
# train a Linear SVM on the data
data = np.load('train_data.npy')
labels = np.load('train_label.npy')   
svclassifier = LinearSVC()  
h = svclassifier.fit(data, labels.ravel())  

In [23]:
acc = [] 
# loop over the testing images
for i in range(0,x_train.shape[0]):
    img = x_test[i]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist = describe(gray)
    prediction = h.predict(hist.reshape(1, -1))
    acc.append(prediction == y_test[i])

In [24]:
np.mean(acc)


0.2342

In [25]:
np.sum(acc*1)

2342

In [36]:
x_test.shape

(10000, 32, 32, 3)